In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tensorflow_text
import spacy
import numpy

In [2]:
header_list=["sentiment","text"]
df = pd.read_csv("test.csv",names=header_list)
df=df.sample(n=10000)
#df=df.drop(["news_title","url","reddit_title"],axis=1)

In [3]:
print(df)

        sentiment                                               text
248020          1  Gorgeous book! Am a fan of Mr. Kylloe's books ...
132369          1  These pearls have been good in the past for us...
360023          1  I saw this at a theater a week or two ago, at ...
46193           0  After reading the book myself, I would not sen...
91200           1  I am as always impressed with the sound and or...
...           ...                                                ...
27853           1  A slightly screwy love affair, between a man, ...
36479           0  I bought this ebook for my son to use on my ki...
273994          1  I love its features: The analog, the digital, ...
63556           0  ITS ALBUM THAT DOES NOT FEATURE MADONNAI BOUGH...
14198           1  A few friends bought the 600 when it first cam...

[10000 rows x 2 columns]


In [4]:
texts=df["text"]
print(texts)
nlp = spacy.load("en_core_web_sm")

248020    Gorgeous book! Am a fan of Mr. Kylloe's books ...
132369    These pearls have been good in the past for us...
360023    I saw this at a theater a week or two ago, at ...
46193     After reading the book myself, I would not sen...
91200     I am as always impressed with the sound and or...
                                ...                        
27853     A slightly screwy love affair, between a man, ...
36479     I bought this ebook for my son to use on my ki...
273994    I love its features: The analog, the digital, ...
63556     ITS ALBUM THAT DOES NOT FEATURE MADONNAI BOUGH...
14198     A few friends bought the 600 when it first cam...
Name: text, Length: 10000, dtype: object


In [5]:
tokenized_texts=[]
for text in texts:
    doc=nlp(text)
    filtered_tokens = [token for token in doc if not token.is_stop if not token.is_punct]
    tokenized_texts.append(filtered_tokens)

In [6]:
print(tokenized_texts[1])

[pearls, good, past, use, tapioca, pudding, goes, fast, house, easy, soak, simple, use, problem, recipe, calls, egg, manner, find, gets, chickeny, smelling, tapioca, pudding, finished, product, finally, cold, suggest, looking, tapioca, pudding, recipe]


In [7]:
max=0
for text in tokenized_texts:
    if len(text)>max:
        max=len(text)
print(max)

128


In [8]:
max=256
expanded_tokenized_texts=[]
for text in tokenized_texts:
    text=text[:max] + [nlp("null")]*(max - len(text))
    expanded_tokenized_texts.append(text)

In [9]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  df.sentiment.to_numpy().reshape(-1, 1)
)

In [10]:
normalized_texts=[]
for text in  expanded_tokenized_texts:
    vectors=[]
    for word in text:
        vectors.append((numpy.array(word.vector)))
    normalized_texts.append(numpy.array(vectors))

In [11]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
train_news, test_news, y_train, y_test =\
  train_test_split(
    normalized_texts, 
    type_one_hot, 
    test_size=.1, 
    random_state=RANDOM_SEED
  )

In [12]:
train_news=np.array(train_news)
test_news=np.array(test_news)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [13]:
print(train_news[1].shape)
print(test_news.shape)
print(y_train.shape)
print(y_test.shape)

(256, 96)
(1000, 256, 96)
(9000, 2)
(1000, 2)


In [14]:
model = keras.models.load_model('./Model/')

2021-11-04 22:30:56.048070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 22:30:56.060828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 22:30:56.061589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 22:30:56.062635: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [15]:
model.evaluate(train_news, y_train)

2021-11-04 22:30:56.990285: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 884736000 exceeds 10% of free system memory.
2021-11-04 22:30:57.583710: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 884736000 exceeds 10% of free system memory.
2021-11-04 22:30:58.058340: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


282/282 [==============================] - 41s 4ms/step - loss: 0.4157 - accuracy: 0.8107


[0.4157377779483795, 0.8106666803359985]

In [16]:
model.evaluate(test_news, y_test)

32/32 [==============================] - 0s 3ms/step - loss: 0.3931 - accuracy: 0.8310


[0.39309772849082947, 0.8309999704360962]

In [17]:
model.save('./Model82/')

2021-11-04 22:51:51.697349: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Model82/assets
